In [2]:
import pandas as pd
import pyspark as ps
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SnowflakeDataLoad").getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/07 17:23:27 WARN Utils: Your hostname, codespaces-ee7d44, resolves to a loopback address: 127.0.0.1; using 10.0.1.87 instead (on interface eth0)
26/02/07 17:23:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/07 17:23:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import os

path = "/workspaces/snowflake-hands-on-aws-project/Data/netflix_titles.csv"
print(os.path.exists(path))


True


### First Dataset

In [10]:
path1 = "/workspaces/snowflake-hands-on-aws-project/Data/apple_stock.csv"
pdf1 = pd.read_csv(path1)
df1 = spark.createDataFrame(pdf1)
df1.show()

+----------+------------------+------------------+------------------+------------------+------------------+---------+
|Unnamed: 0|         Adj Close|             Close|              High|               Low|              Open|   Volume|
+----------+------------------+------------------+------------------+------------------+------------------+---------+
|1980-12-12|0.0988344699144363|0.1283479928970337|0.1289059966802597|0.1283479928970337|0.1283479928970337|469033600|
|1980-12-15|0.0936782136559486|0.1216519996523857|0.1222100034356117|0.1216519996523857|0.1222100034356117|175884800|
|1980-12-16|0.0868024080991745|0.1127230003476142|0.1132809966802597|0.1127230003476142|0.1132809966802597|105728000|
|1980-12-17|0.0889508724212646|0.1155129969120025|0.1160710006952285|0.1155129969120025|0.1155129969120025| 86441600|
|1980-12-18|0.0915297642350196|0.1188620030879974|0.1194199994206428|0.1188620030879974|0.1188620030879974| 73449600|
|1980-12-19|0.0971156954765319|0.1261159926652908|0.1266

Traceback (most recent call last):
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
BrokenPipeError: [Errno 32] Broken pipe


In [14]:
df1.printSchema()
# pdf1.isnull().sum()

root
 |-- Unnamed: 0: string (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: long (nullable = true)



### Second Dataset

In [5]:

# Initialize Spark
spark.sparkContext.setLogLevel("ERROR")
'''
# Read CSV with pandas
path = "/workspaces/snowflake-hands-on-aws-project/Data/netflix_titles.csv"
pdf = pd.read_csv(path)
'''
# Convert to Spark DataFrame
df = spark.createDataFrame(pdf)

df.show(3)


+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                 nan|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|            nan|Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|          nan|Septem

Traceback (most recent call last):
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
BrokenPipeError: [Errno 32] Broken pipe


In [21]:
from pyspark.sql import functions as F

# Fill missing string columns
df2 = df2.fillna({
    "director": "Unknown",
    "cast": "Unknown",
    "country": "Unknown",
    "rating": "Not Rated",
    "duration": "Unknown"
})

#  Convert date_added to proper Date type
df2 = df2.withColumn(
    "date_added",
    F.to_date(F.col("date_added"), "MMMM d, yyyy")
)

#  Handle remaining null date values
df2 = df2.fillna({
    "date_added": "1900-01-01"
})

#  Trim string columns (very important in real projects)
string_cols = ["type", "title", "director", "cast", "country", "rating", "listed_in"]




+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+----------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                 nan|United States|2021-09-25|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|            nan|Ama Qamata, Khosi...| South Africa|2021-09-24|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|          nan|2021-09-24|        2021| TV-MA| 1 Season|Crime

Traceback (most recent call last):
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
BrokenPipeError: [Errno 32] Broken pipe


In [36]:


df2.show(3)

+-------+-------+--------------------+---------------+--------------------+-------------+-------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|         date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+-------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|             Unknown|United States|2021-09-25 00:00:00|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|        Unknown|Ama Qamata, Khosi...| South Africa|2021-09-24 00:00:00|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|          nan|2

Traceback (most recent call last):
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
BrokenPipeError: [Errno 32] Broken pipe


In [39]:
df2.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: timestamp (nullable = true)
 |-- release_year: long (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [48]:
# Save to CSV
df2.write.csv("netflix_cleaned.csv", header=True)


UnsupportedOperationException: getSubject is not supported

### Third Dataset

In [50]:
path2 = "/workspaces/snowflake-hands-on-aws-project/Data/amazon_products_sales_data_cleaned.csv"
pdf3 = pd.read_csv(path2)
df3 = spark.createDataFrame(pdf3)
df3.show()

+--------------------+--------------+-------------+--------------------+----------------+--------------+--------------+------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|       product_title|product_rating|total_reviews|purchased_last_month|discounted_price|original_price|is_best_seller|is_sponsored|          has_coupon|buy_box_availability|delivery_date| sustainability_tags|   product_image_url|    product_page_url|  data_collected_at|   product_category|discount_percentage|
+--------------------+--------------+-------------+--------------------+----------------+--------------+--------------+------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+
|BOYA BOYALINK 2 W...|           4.6|        375.0|             

Traceback (most recent call last):
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 233, in manager
    code = worker(sock, authenticated)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/snowflake-hands-on-aws-project/.venv/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 87, in worker
    outfile.flush()
BrokenPipeError: [Errno 32] Broken pipe


In [53]:
pdf3.isnull().sum()

product_title               0
product_rating           1024
total_reviews            1024
purchased_last_month    10511
discounted_price         2062
original_price           2062
is_best_seller              0
is_sponsored                0
has_coupon                  0
buy_box_availability    14653
delivery_date           11983
sustainability_tags     39267
product_image_url           0
product_page_url         2069
data_collected_at           0
product_category            0
discount_percentage      2062
dtype: int64

In [58]:
df3.printSchema()

root
 |-- product_title: string (nullable = true)
 |-- product_rating: double (nullable = false)
 |-- total_reviews: double (nullable = false)
 |-- purchased_last_month: double (nullable = false)
 |-- discounted_price: double (nullable = false)
 |-- original_price: double (nullable = false)
 |-- is_best_seller: string (nullable = true)
 |-- is_sponsored: string (nullable = true)
 |-- has_coupon: string (nullable = true)
 |-- buy_box_availability: string (nullable = false)
 |-- delivery_date: string (nullable = false)
 |-- sustainability_tags: string (nullable = false)
 |-- product_image_url: string (nullable = true)
 |-- product_page_url: string (nullable = false)
 |-- data_collected_at: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- discount_percentage: double (nullable = false)



In [54]:
# Step 1: Fill numeric nulls
numeric_fill = {
    "product_rating": 0,
    "total_reviews": 0,
    "purchased_last_month": 0,
    "discounted_price": 0,
    "original_price": 0,
    "discount_percentage": 0
}
df3 = df3.fillna(numeric_fill)

In [ ]:
# Step 2: Fill string/object nulls
string_fill = {
    "buy_box_availability": "Unknown",
    "delivery_date": "Unknown",
    "sustainability_tags": "Unknown",
    "product_page_url": "Unknown",
    "buy_box_availability": "Unknown",
    "product_page_url": "Unknown",
    "sustainability_tags": "Unknown",
    "delivery_date ": "Unknown"
    
    
}
df3 = df3.fillna(string_fill)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `delivery_date ` cannot be resolved. Did you mean one of the following? [`product_title`, `product_rating`, `total_reviews`, `purchased_last_month`, `discounted_price`, `original_price`, `is_best_seller`, `is_sponsored`, `has_coupon`, `buy_box_availability`, `delivery_date`, `sustainability_tags`, `product_image_url`, `product_page_url`, `data_collected_at`, `product_category`, `discount_percentage`]. SQLSTATE: 42703

In [61]:
df3.printSchema()
df3.count()



root
 |-- product_title: string (nullable = true)
 |-- product_rating: double (nullable = false)
 |-- total_reviews: double (nullable = false)
 |-- purchased_last_month: double (nullable = false)
 |-- discounted_price: double (nullable = false)
 |-- original_price: double (nullable = false)
 |-- is_best_seller: string (nullable = true)
 |-- is_sponsored: string (nullable = true)
 |-- has_coupon: string (nullable = true)
 |-- buy_box_availability: string (nullable = false)
 |-- delivery_date: string (nullable = false)
 |-- sustainability_tags: string (nullable = false)
 |-- product_image_url: string (nullable = true)
 |-- product_page_url: string (nullable = false)
 |-- data_collected_at: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- discount_percentage: double (nullable = false)



42675

In [63]:
# Save all three datasets

df3.toPandas().to_csv("amazon_products_cleaned.csv", index=False)


print("All datasets saved successfully!")

All datasets saved successfully!


In [3]:
import pandas as pd
pt ="/workspaces/snowflake-hands-on-aws-project/Data/netflix_cleaned.csv"
dt = pd.read_csv(pt)
dt.info()

<class 'pandas.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   show_id       8807 non-null   str  
 1   type          8807 non-null   str  
 2   title         8807 non-null   str  
 3   director      8807 non-null   str  
 4   cast          8807 non-null   str  
 5   country       7976 non-null   str  
 6   date_added    8709 non-null   str  
 7   release_year  8807 non-null   int64
 8   rating        8807 non-null   str  
 9   duration      8807 non-null   str  
 10  listed_in     8807 non-null   str  
 11  description   8807 non-null   str  
dtypes: int64(1), str(11)
memory usage: 825.8 KB
